<a href="https://colab.research.google.com/github/Rijan-Joshi/Fun-Learning/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [31]:
#Importing the data

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    transform = ToTensor(),
    download= True
)

test_data = datasets.FashionMNIST(
    root='data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [32]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

X, y = next(iter(test_dataloader))
X.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [40]:
#Creating Model in PyTorch

device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
print(f"Using {device} device")

#Defining the model

class NeuralNetwork(nn.Module):

  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x) #Raw Output
    return logits


Using cpu device


In [41]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [42]:
#Loss Functions and Optimizers
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [43]:
#Start training

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


    if batch % 100 == 0:
      loss, current = loss.item(), (batch +1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [46]:
#Define testing function

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)

      #Prediction
      y_pred = model(X)
      test_loss += loss_fn(y_pred, y).item()
      correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  accuracy = correct / size

  print(f'Test Error: \n Accuracy {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n')



In [47]:
#Start Training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.185740  [   64/60000]
loss: 2.180510  [ 6464/60000]
loss: 2.123471  [12864/60000]
loss: 2.139266  [19264/60000]
loss: 2.110494  [25664/60000]
loss: 2.040173  [32064/60000]
loss: 2.069142  [38464/60000]
loss: 1.996860  [44864/60000]
loss: 2.010046  [51264/60000]
loss: 1.939097  [57664/60000]
Test Error: 
 Accuracy 57.3%, Avg loss: 1.939346 

Epoch 2
-------------------------------
loss: 1.970535  [   64/60000]
loss: 1.947326  [ 6464/60000]
loss: 1.835871  [12864/60000]
loss: 1.866160  [19264/60000]
loss: 1.784335  [25664/60000]
loss: 1.719348  [32064/60000]
loss: 1.729916  [38464/60000]
loss: 1.636988  [44864/60000]
loss: 1.661820  [51264/60000]
loss: 1.555595  [57664/60000]
Test Error: 
 Accuracy 59.7%, Avg loss: 1.573892 

Epoch 3
-------------------------------
loss: 1.637090  [   64/60000]
loss: 1.606931  [ 6464/60000]
loss: 1.458247  [12864/60000]
loss: 1.518823  [19264/60000]
loss: 1.415537  [25664/60000]
loss: 1.394611  [32064/60000

In [51]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[7][0], test_data[7][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  pred_idx = pred[0].argmax()
  print(f"Predicted: {classes[pred_idx]}, Actual: {classes[y]}")



Predicted: Coat, Actual: Shirt
